In [2]:
import pandas as pd
import json
import time
import numpy as np
import random
import os
from tqdm import tqdm
#import cv2 

import matplotlib.pyplot as plt

from moviepy.editor import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip, ffmpeg_resize

In [2]:
good_df = pd.read_csv("../questions.csv")
good_clip_set = set(good_df["clip_name"])

In [14]:
frames = 10
vid_location = ""#specify the locations
clip_dir = f"{vid_location}/clips"
frames_dir = f"{vid_location}/frames_{frames}"
if not os.path.exists(frames_dir):
    os.mkdir(frames_dir)
uploaded_videos = [vid for vid in os.listdir(frames_dir) if "ipynb" not in vid and len([frame for frame in os.listdir(f"{frames_dir}/{vid}") if "ipynb" not in frame]) == 10]
batch_videos = [vid[:vid.find(".")] for vid in os.listdir(clip_dir) if "ipynb" not in vid]

In [15]:
import multiprocessing
from multiprocessing import Queue as multi_queue
import time

def task(the_queue):
    while True:
        if the_queue.empty():
            break
            
        vid_name = the_queue.get()
        vid_path = f"{clip_dir}/{vid_name}.mp4"
        is_it_done = True
        if vid_name in uploaded_videos:
            continue
        vid_folder = f"{frames_dir}/{vid_name}"        
        if not os.path.exists(vid_folder):
            os.mkdir(vid_folder)
        uploaded_frames = os.listdir(vid_folder)
        for i in range(frames):
            if f"{i}.jpg" not in uploaded_frames:
                is_it_done = False
        
        if is_it_done:
            continue

        try:
            video = VideoFileClip(vid_path)
        except:
            continue
        vid_frames = [f for f in video.iter_frames()]
        frames_sec = np.linspace(0, len(vid_frames)-1, num=frames)
        
        for f_i in range(len(frames_sec)):
            f = frames_sec[f_i]
            frame = vid_frames[int(f)]
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            cv2.imwrite(f"{vid_folder}/image_{f_i+1}.jpg", frame)

def progress_task(initially_done, the_queue, to_do):
    global done_trigger
    print("Done: -1%. Time left: -1 seconds", flush=True, end='\r')
            
    start = time.time()
    while not the_queue.empty():
        time.sleep(1)
        done = to_do - the_queue.qsize()
            
        speed = done / (time.time() - start)
        rest = to_do - done
            
        if speed == 0:
            time_left = -1
        else:
            time_left = rest / speed
        
        print(f"Done: {100 * (initially_done + done) / (initially_done+to_do)}% Time left: {time_left / 3600} seconds", end='\r', flush = True)

the_queue = multi_queue()
for video_name in batch_videos:
    if video_name not in good_clip_set:
        continue
    if video_name not in uploaded_videos:
        the_queue.put(video_name)
        

initially_done = len(uploaded_videos)
to_do = len(batch_videos) - initially_done
progress_process = multiprocessing.Process(target=progress_task, args=(initially_done, the_queue, to_do))

procceses = []
for i in range(20):
    p1 = multiprocessing.Process(target=task, args=(the_queue,))
    procceses.append(p1)
    
progress_process.start()
for p in procceses:
    p.start()
    
for p in procceses:
    p.join()
done_trigger = True
progress_process.join()